In [1]:
import pandas as pd
import openpyxl as opx
import numpy as np
import seaborn as sns
from fuzzywuzzy import fuzz
import string

C:\Users\chris\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# create dict + function for error messages

error_msg = {}

def errormessage(summary, key, message):
    if key in summary.keys():
        error_msg[str(key)].append(str(message))
    else:
        error_msg[str(key)] = [str(message)]
        
print(error_msg)

{}


In [3]:
# load time series

ts = pd.read_excel("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\test_ts.xlsx", sheet_name=None)

#rename sheets

try:
    ts['Anonymised Data'] = ts['Sheet1']
except:
    pass

try:
    ts['Anonymised Data'] = ts['Sheet 1']
except:
    pass

# load patient data and clean

p = pd.read_excel("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\test_p.xlsx", sheet_name=None)

try:
    patient = pd.DataFrame(data=p['Sheet1'])
except:
    print('The sheet isnt called Sheet1, rename this')
    
patient.columns = patient.iloc[2]
patient = patient.iloc[3:]
patient.columns.name = None
patient = patient.set_index(patient['sid_number']).iloc[:, 1:]
patient.index.name = None
if patient.columns[0] == 'hospital_number':
    patient = patient.iloc[:, 1:]
sid_count = 0
for SID in patient['sid_number']:
    if not pd.isnull(SID):
        sid_count += 1
patient = patient.iloc[:sid_count]
patient.fillna(value=0, inplace=True)
patient

,sid_number,age,gender_male,gender_female,gender_non_binary,asian,black,mixed,white,other,...,refer_other_text,over_8,1-7_days,otd,proposed_lvl,actual_lvl,difference_nobed,difference_other,difference_other_text,difference_unknown
XYZ015,XYZ015,50,1,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
XYZ016,XYZ016,51,0,1,0,0,0,0,1,0,...,0,1,0,0,1,1,0,0,0,0
XYZ017,XYZ017,75,1,0,0,0,0,0,1,0,...,0,0,1,0,1,1,0,0,0,0
XYZ018,XYZ018,73,0,1,0,0,0,0,1,0,...,0,0,1,0,1,1,0,0,0,0
XYZ0117,XYZ0117,55,0,1,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYZ01181,XYZ01181,24,1,0,0,0,0,0,1,0,...,0,1,0,0,1,1,0,0,0,0
XYZ01182,XYZ01182,82,1,0,0,0,0,0,1,0,...,0,1,0,0,1,1,0,0,0,0
XYZ01189,XYZ01189,65,1,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,1,Vasopressor requirement,0
XYZ01192,XYZ01192,62,0,1,0,0,0,0,1,0,...,0,1,0,0,1,1,0,0,0,0


In [4]:
# check for mismatch in time series

time_series = pd.DataFrame(data=ts['Anonymised Data'])
mismatch = time_series.iloc[94, 5:18]
mis1 = mismatch.sum(axis=0)

if mis1 > 0:
    errormessage(error_msg, 'Input Data', 'There is an issue with the format of the Input Data tab of the Time-Series CRF. Usually this issue arises from extra spaces or commas in the Input Data tab causing a mismatch in the way hospital IDs are anonymised.')

In [5]:
# strip time series dataframe of counts for interaction checking

time_series.columns = time_series.iloc[0]
time_series = time_series.iloc[1:92, 18:31]
time_series

,otd_cancel,otd_cancel_nobed,ref_lvl1,plan_adm_lvl1,adm_lvl1,ref_lvl2,plan_adm_lvl2,adm_lvl2,adm_7d_lvl2,ref_lvl3,plan_adm_lvl3,adm_lvl3,adm_7d_lvl3
1,"XYZ011, XYZ012, XYZ013, XYZ014",NaN,"XYZ015, XYZ016, XYZ017, XYZ018","XYZ018, XYZ016, XYZ017","XYZ018, XYZ016, XYZ017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"XYZ019, XYZ0110",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,XYZ0111,NaN,NaN,XYZ0112,XYZ0112,NaN,NaN,NaN,XYZ0113,NaN,NaN,NaN,XYZ0114
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"XYZ0112, XYZ0115",NaN,NaN,NaN,NaN
5,XYZ0116,XYZ0116,NaN,NaN,NaN,XYZ0117,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,XYZ01191,NaN,XYZ01192,XYZ01192,XYZ01192,NaN,NaN,NaN,XYZ01193,NaN,NaN,NaN,NaN
88,"XYZ01194, XYZ01195",NaN,XYZ01196,XYZ01196,XYZ01196,XYZ01197,XYZ01197,"XYZ01197, XYZ01198, XYZ01199, XYZ01200","XYZ01197, XYZ01198, XYZ01199, XYZ01200",NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,XYZ01201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# create list of hospital numbers in each cell

def listify(cell):
    if pd.isna(cell):
        return []
    else:
        cell = str(cell)
        cell = cell.replace(" ", "")
        listy = list(cell.split(","))
        return listy

time_series = time_series.applymap(listify)
time_series

,otd_cancel,otd_cancel_nobed,ref_lvl1,plan_adm_lvl1,adm_lvl1,ref_lvl2,plan_adm_lvl2,adm_lvl2,adm_7d_lvl2,ref_lvl3,plan_adm_lvl3,adm_lvl3,adm_7d_lvl3
1,"[XYZ011, XYZ012, XYZ013, XYZ014]",[],"[XYZ015, XYZ016, XYZ017, XYZ018]","[XYZ018, XYZ016, XYZ017]","[XYZ018, XYZ016, XYZ017]",[],[],[],[],[],[],[],[]
2,[],[],"[XYZ019, XYZ0110]",[],[],[],[],[],[],[],[],[],[]
3,[XYZ0111],[],[],[XYZ0112],[XYZ0112],[],[],[],[XYZ0113],[],[],[],[XYZ0114]
4,[],[],[],[],[],[],[],[],"[XYZ0112, XYZ0115]",[],[],[],[]
5,[XYZ0116],[XYZ0116],[],[],[],[XYZ0117],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,[XYZ01191],[],[XYZ01192],[XYZ01192],[XYZ01192],[],[],[],[XYZ01193],[],[],[],[]
88,"[XYZ01194, XYZ01195]",[],[XYZ01196],[XYZ01196],[XYZ01196],[XYZ01197],[XYZ01197],"[XYZ01197, XYZ01198, XYZ01199, XYZ01200]","[XYZ01197, XYZ01198, XYZ01199, XYZ01200]",[],[],[],[]
89,[],[],[],[],[],[],[],[],[],[],[],[],[]
90,[XYZ01201],[],[],[],[],[],[],[],[],[],[],[],[]


In [7]:
# check if site code has been added to hospital numbers

for row in time_series.iterrows():
    row_data = row[1]
    for item in row_data:
        if not item:
            pass
        else:
            if item[0].isdigit():
                errormessage(error_msg, 'Hospital Details', 'No hospital code has been added on the Hospital Details tab, please add one so that full study IDs can be generated. Once this is done, please ensure that the SIDs on the EhC referrals tab of the Time Series match up with the SIDs in the Patient CRF')
                break
            break
        break
    break
    
print(error_msg)

{}


In [ ]:
# iterate through rows of time series to conduct sense checks
for row in time_series.iterrows():
    row_data = row[1]
    
    # check if all cancellations due to a lack of ECB are in cancellations and ECB referrals

    # check if cancellations due to a lack of ECB is empty
    if row_data[1] != []:
        for SID in row_data[1]:
            
            # cancellations
            
            # check if cancellation field is empty or if SID appears in cancellations on that date
            if row_data[0] == [] or SID not in row_data[0]:
                errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of cancellations due to lack of ECB, but not in total cancellations for that day, please add them to the cancellations column on the Input Data tab.')
                
            # referrals to each level
            
            # check if all referral fields are empty on that date
            if (type(row_data[2]) == float and type(row_data[5]) == float and type(row_data[9]) == float):
                errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of cancellations due to lack of ECB, but not in referrals to enhanced care, please add them to the referrals column for the appropriate level on the Input Data tab.')
            
            # check if SID appears in any of the referral fields on that date
            else:
                if SID not in row_data[2] and SID not in row_data[5] and SID not in row_data[9]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of cancellations due to lack of ECB, but not in referrals to enhanced care, please add them to the referrals column for the appropriate level on the Input Data tab.')

    # check that cancellations dont end up admitted that day
    
    # check if cancellations is empty
    if row_data[0] != []:
        for SID in row_data[0]:
            
            # check if SID appears in any of the admission fields on that date
            if SID in row_data[4] or SID in row_data[7] or SID in row_data[11]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of cancellations but is also admitted the same day to a level of enhanced care, please check if this is correct and amend as necessary on the Input Data tab.')
        
    # check that planned admissions are also in referrals, admissions and admissions within 7 days
    
    # level 1
    
    # check if planned admissions to level 1 is empty
    if row_data[3] != []:
        for SID in row_data[3]:
            
            # check if SID appears in the referral field for level 1
            if SID not in row_data[2]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 1, but not in referrals to level 1, please add them to the referrals for level 1 or remove them from planned admissions on the Input Data tab.')
            if SID not in row_data[4]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 1, but not in admissions to level 1 for that day, please add them to the admissions for level 1 or remove them from planned admissions on the Input Data tab.')
    
    # level 2
    
    # check if planned admissions to level 2 is empty
    if row_data[6] != []:
        for SID in row_data[6]:
            
            # check if SID appears in the referral field for level 2
            if SID not in row_data[5]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 2, but not in referrals to level 2, please add them to the referrals for level 2 or remove them from planned admissions on the Input Data tab.')
            if SID not in row_data[7]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 2, but not in admissions to level 2 for that day, please add them to the admissions for level 2 or remove them from planned admissions on the Input Data tab.')
            if SID not in row_data[8]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 2, but not in admissions to level 2 within 7 days, please add them to the admissions for level 2 within 7 days [all admisisons on day 1 also should be in the within 7 days column] or remove them from planned admissions on the Input Data tab.')
    
    # level 3
    
    # check if planned admissions to level 3 is empty
    if row_data[10] != []:
        for SID in row_data[10]:
            
            # check if SID appears in the referral field for level 3
            if SID not in row_data[9]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 3, but not in referrals to level 3, please add them to the referrals for level 3 or remove them from planned admissions on the Input Data tab.')
            if SID not in row_data[11]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 3, but not in admissions to level 3 for that day, please add them to the admissions for level 3 or remove them from planned admissions on the Input Data tab.')
            if SID not in row_data[12]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of planned admissions to level 3, but not in admissions to level 3 within 7 days, please add them to the admissions for level 3 within 7 days [all admisisons on day 1 also should be in the within 7 days column] or remove them from planned admissions on the Input Data tab.')
     
    # check that admissions immediately after surgery appear in admissions within 7 days
    # and check if admissions match up with correct predicted level of enhanced care on patient CRF
    
    #level 1
    
    # check if admissions to level 1 is empty
    if row_data[4] != []:
        for SID in row_data[4]:
            
            # check if actual level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-5] != 1:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of admissions to level 1, but the actual post-op level of care in the patient CRF is different from this, please check this and amend as necessary.')
            except:
                pass
    
    # level 2
    
    # check if admissions to level 2 is empty
    if row_data[7] != []:
        for SID in row_data[7]:
            
            # check if SID appears in admissions within 7 days for level 2
            if SID not in row_data[8]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of admissions to level 2 immediately post-op, but not admissions within 7 days to level 2, please add them to admissions to level 2 within 7 days on the Input Data tab.')

            # check if actual level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-5] != 2:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of admissions to level 2, but the actual post-op level of care in the patient CRF is different from this, please check this and amend as necessary.')
            except:
                pass
            
    # level 3
    
    # check if admissions to level 3 is empty
    if row_data[11] != []:
        for SID in row_data[11]:
            
            # check if SID appears in admissions within 7 days for level 3
            if SID not in row_data[12]:
                    errormessage(error_msg, 'Anoynmised Data', 'Patient with SID '+str(SID)+' is in the list of admissions to level 3 immediately post-op, but not admissions within 7 days to level 3, please add them to admissions to level 3 within 7 days on the Input Data tab.')
            
            # check if actual level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-5] != 3:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of admissions to level 3, but the actual post-op level of care in the patient CRF is different from this, please check this and amend as necessary.')
            except:
                pass
    
    
    # check if all referrals are only made one per day and end up in either admissions, cancellations or ward stepdowns
    # and check if referrals match up with correct predicted level of enhanced care on patient CRF
    
    # level 1
    
    # check if referrals to level 1 is empty
    if row_data[2] != []:
        for SID in row_data[2]:
            
            # check if SID appears in any of the other referral fields on that date
            if SID in row_data[5] or SID in row_data[9]:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 1, but is also referred to another level of enhanced care that day, please check this and amend as necessary.')
            
            # check if SID appears in any of the admission fields on that date or in cancellations
            if SID not in row_data[4] and SID not in row_data[7] and SID not in row_data[11] and SID not in row_data[0]:
                try:
                    if patient.loc[SID][-5] > 0:
                        errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 1, but is not admitted to any level of enhanced care, is not listed as cancelled, and is not listed as a ward stepdown in the Patient CRF, please check this and amend as necessary.')
                except:
                    pass
                
            # check if predicted level of enhanced care is level 1 in patient CRF
            try:
                if patient.loc[SID][-6] != 1:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 1, but the predicted level of care in the patient CRF is different from this, please check this and amend as necessary.')
            except:
                pass
    
    # level 2
    
    # check if referrals to level 2 is empty
    if row_data[5] != []:
        for SID in row_data[5]:
            
            # check if SID appears in any of the other referral fields on that date
            if SID in row_data[2] or SID in row_data[9]:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 2, but is also referred to another level of enhanced care that day, please check this and amend as necessary.')
            
            # check if SID appears in any of the admission fields on that date or in cancellations
            if SID not in row_data[4] and SID not in row_data[7] and SID not in row_data[11] and SID not in row_data[0]:
                try:
                    if patient.loc[SID][-5] > 0:
                        errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 2, but is not admitted to any level of enhanced care, is not listed as cancelled, and is not listed as a ward stepdown in the Patient CRF, please check this and amend as necessary.')
                except:
                    pass
                
            # check if predicted level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-6] != 2:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 2, but the predicted level of care in the patient CRF is different from this, please check this and amend as necessary.')
            except:
                pass
            
    # level 3
    
    # check if referrals to level 3 is empty
    if row_data[9] != []:
        for SID in row_data[9]:
            
            # check if SID appears in any of the other referral fields on that date
            if SID in row_data[2] or SID in row_data[5]:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 3, but is also referred to another level of enhanced care that day, please check this and amend as necessary.')
            
            # check if SID appears in any of the admission fields on that date or in cancellations
            if SID not in row_data[4] and SID not in row_data[7] and SID not in row_data[11] and SID not in row_data[0]:
                try:
                    if patient.loc[SID][-5] > 0:
                        errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 3, but is not admitted to any level of enhanced care, is not listed as cancelled, and is not listed as a ward stepdown in the Patient CRF, please check this and amend as necessary.')
                except:
                    pass
                
            # check if predicted level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-6] != 3:
                    errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is in the list of referrals to level 3, but the predicted level of care in the patient CRF is different from this, please check this and amend as necessary.')        
            except:
                pass
            
print(error_msg)

In [ ]:
# check if all referrals to ehanced care turn up in the patient CRF

EhC_refs = []

for cell in time_series['ref_lvl1']:
    EhC_refs = EhC_refs + cell
    
for cell in time_series['ref_lvl2']:
    EhC_refs = EhC_refs + cell
    
for cell in time_series['ref_lvl3']:
    EhC_refs = EhC_refs + cell
    
missing_SIDs = []
for SID in EhC_refs:
    if SID not in patient['sid_number']:
        missing_SIDs += [SID]
        errormessage(error_msg, 'Patient CRF', 'The following SIDs are identified as referrals to enhanced care on the time series but do not appear in the Patient CRF, please add these: '+str(missing_SIDs))

print(error_msg)

In [ ]:
# check if there are any double instances in the patient CRF list

SID_list = list(patient['sid_number'])
SID_unique = set(SID_list)
if len(SID_list) != len(SID_unique):
    seen = set()
    duplicates = []
    for SID in SID_list:
        if SID in seen:
            duplicates.append(SID)
        else:
            seen.add(SID)
    errormessage(error_msg, 'Patient CRF', 'The following SIDs are duplicated in the patient CRF, which may be correct if they had two admissions during the study period, please check these: '+str(duplicates))

print(error_msg)

In [ ]:
# iterate through rows of patient CRF to conduct sense checks
for row in patient.iterrows():
    row_data = row[1]
    
    # check if age is >18
    if row_data[1] < 18:
        SID = row_data[0]
        errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' has an age listed that is <18yrs, please check this and amend as necessary.')
    
    # check if only 1 gender field flagged
    if row_data[2] + row_data[3] + row_data[4] != 1:
        SID = row_data[0]
        errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' has an error in their gender denomination (either more than 1 selected or none at all), please check this and amend as necessary.')
    
    # check if only 1 ethnicity field flagged
    if row_data[5] + row_data[6] + row_data[7] + row_data[8] + row_data[9] != 1:
        SID = row_data[0]
        errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' has an error in their ethnicity denomination (either more than 1 selected or none at all), please check this and amend as necessary.')
 
    # check if all those flagged as cancellations appear in cancellation column of time series and have a reason for cancellation
    
    # check if cancellation is flagged in patient CRF
    if row_data[31] == 1:
        
        # check if this SID appears in the cancellation field in the time series
        SID = row_data[0]
        count = 0
        for cell in time_series['otd_cancel']:
            if SID in cell:
                count += 1
                break
        if count == 0:
            errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is flagged as a cancellation in the patient CRF but does not appear in the time series as a cancellation on any day, please check this and amend as necessary.')
        
        # check if reason a is flagged
        if row_data[42] != 1 and row_data[43] != 1:
            errormessage(error_msg, 'Patient CRF', 'Patient with SID '+str(SID)+' is flagged as a cancellation in the patient CRF but does not have a reason listed, either they were cancelled due to a lack of enhanced care bed OR this was for another reason, please flag one of these parameters.')
        
    # check if all those flagged as cancellations due to a lack of ECB appear in cancellation due to lack of ECB column of time series

    # check if cancellation due to lack of ECB is flagged in patient CRF
    if row_data[42] == 1:
        
        # check if this SID appears in the cancellation due to lack of ECB field in the time series
        SID = row_data[0]
        count = 0
        for cell in time_series['otd_cancel_nobed']:
            if SID in cell:
                count += 1
                break
        if count == 0:
            errormessage(error_msg, 'Anoynmised Data or Patient CRF', 'Patient with SID '+str(SID)+' is flagged as a cancellation due to a lack of enhanced care bed in the patient CRF but does not appear in the time series as a cancellation for this reason on any day, please check this and amend as necessary.')
         
    # check if there is a reason given for a difference between predicted and actual post-op destination
    
    # check if there is a difference between predicted and actual
    if row_data[-6] != row_data[-5]:
        SID = row_data[0]
        if row_data[-4] != 1 and row_data[-3] != 1 and row_data[-1] != 1:
            errormessage(error_msg, 'Patient CRF', 'Patient with SID '+str(SID)+' was admitted to a different level of critical care than was predicted but there is no reason listed, either this was due to a lack of enhanced care bed OR this was for another reason, please flag one of these parameters.')
    
print(error_msg)

In [ ]:
for key, value in error_msg.items():
    print(key)
    for item in value:
        print(item)
        print('\n')